# Accessing Data Feeds in Ascend from Python

This notebook demonstrates how to use the 
[Ascend SDK for Python](https://github.com/ascend-io/ascend-python-sdk) to access Data Feed.

## Configuration

> If you are not already an Ascend user you can [Request a Free Trial](https://www.ascend.io/get-started/)

This notebook will connect to your Ascend environment via a Web API using a Service Account you create in the Ascend UI.

If you are using the `trial.ascend.io` environment, you can leave this configuration as-is.
Otherwise, change `host` to the hostname of your Ascend environment,
and (optionally) modify the `profile` under which your credentials are stored.


In [1]:
host = "trial.ascend.io"
profile = "trial"

## Credentials

To run this notebook, access Data Feeds from Python, you will need

  * a Service Account with `DATA FEED READ ONLY` permission
  * an Access Key ID and Secret for that Service Account

You can create these in the Ascend UI by going to **Data Service > Service Accounts**.

Access Keys should not be stored in a notebook. 
Instead, this notebook will look for them in `~/.ascend/credentials` on the machine where your Jupyter server is running.
Your `~/.ascend/credentials` file should look like this (substitute your Ascend Access Key ID and Secret Access Key):
```
[trial]
ascend_access_key_id=Y0URACC355K3Y1D
ascend_secret_access_key=yourSecret!AccessKeyisthelong1
```

Once you have a `credentials` file, you can read it with `configparser` and
create a `Client` to connect to the host using your credentials.

In [2]:
from ascend.client import Client
import configparser
import os


config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.ascend/credentials"))

access_id = config.get(profile, "ascend_access_key_id")
secret_key = config.get(profile, "ascend_secret_access_key")

A = Client(host, access_id, secret_key)
A

## List Available Data Feeds

The Data Feed API gives you access to all Data Feeds either produced by your Data Service
or shared with your Data Service by others.

Use `list_data_feeds` to list all the Data Feeds which you can access.
The optional argument allows you to filter this list to one data service.


In [3]:
A.list_data_feeds()

[<ascend.model.DataFeed Getting_Started_with_Ascend._DF__Clusters_w__Solar>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Usage__Weather__Solar>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Usage__Weather>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Clusters>,
 <ascend.model.DataFeed Getting_Started_with_Ascend.Yield_Curve__Data_Feed_>]

In [4]:
DATA_SERVICE = 'Getting_Started_with_Ascend'
A.list_data_feeds(DATA_SERVICE)

[<ascend.model.DataFeed Getting_Started_with_Ascend._DF__Clusters_w__Solar>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Usage__Weather__Solar>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Usage__Weather>,
 <ascend.model.DataFeed Getting_Started_with_Ascend._DF__Clusters>,
 <ascend.model.DataFeed Getting_Started_with_Ascend.Yield_Curve__Data_Feed_>]

## Connect to a Data Feed 

Use `get_data_feed` to connect to a Data Feed.

You will need the `id` of the Data Service that produces it, and the `id` of the Data Feed itself.
You can find these in the Ascend UI, or you can copy them out of the results of `list_data_feeds`.

In [5]:
feed = A.get_data_feed(DATA_SERVICE, 'Yield_Curve__Data_Feed_')
feed

<ascend.model.DataFeed Getting_Started_with_Ascend.Yield_Curve__Data_Feed_>

## Read the Records

Once you've connected to a Data Feed, you can use `get_records` to read the records from the Data Feed.

The records can be iterated over as they come through the API,
or used to create a Pandas DataFrame.

In [6]:
count = 0
for row in feed.get_records():
    if count % 100 == 0:
        print(".", end='', flush=True)
    count += 1
print("\n{} records".format(count))

.................................................................................................................
11291 records


In [7]:
import pandas as pd
pd.DataFrame.from_records(feed.get_records())

,date_col,yield_val
0,1976-06-01,0.68
1,1976-06-02,0.71
2,1976-06-03,0.70
3,1976-06-04,0.77
4,1976-06-07,0.79
5,1976-06-08,0.79
6,1976-06-09,0.82
7,1976-06-10,0.86
8,1976-06-11,0.83
9,1976-06-14,0.83
